In [3]:
import torch
L1, L2 = 1.0, 1.0 
N = 5  
n = 10   
k1, k2 = 1, 1

def compute_normalization_constant(k1, k2, L1, L2):
    return torch.tensor(((L1 * L2) / 2)**0.5)

x_j = torch.randn(N, n, 2)  

def fourier_basis(x, k1, k2, h_k):
    return (1 / h_k) * torch.cos(k1 * x[:, 0]) * torch.cos(k2 * x[:, 1])

def compute_fourier_coefficients(x_j, k1, k2, L1, L2):
    N_t = x_j.shape[0]  
    n = x_j.shape[1]   
    h_k = compute_normalization_constant(k1, k2, L1, L2)
    c_k = torch.zeros(N_t)

    for j in range(N_t):
        f_k_values = fourier_basis(x_j[j], k1, k2, h_k)
        c_k[j] = torch.sum(f_k_values) / n
    
    return c_k
c_k = compute_fourier_coefficients(x_j, k1, k2, L1, L2)
print("Fourier coefficients c_k(t) for k = (", k1, ",", k2, "):")
print(c_k)


Fourier coefficients c_k(t) for k = ( 1 , 1 ):
tensor([0.1990, 0.3516, 0.8914, 0.7726, 0.8371])


In [16]:
""" function C_t """
import torch

# Function to compute the indicator function I_B(x_j(t_i), epsilon)
def indicator_function(x, x_j_point, epsilon=1e-2):
    distance = torch.norm(x - x_j_point)
    return (distance < epsilon).float()

# Function to compute C_t(x)
def compute_Ct(x, X):
    #X is matrix of computed values X_j shape(N, n)
    C_t_x = 0
    for i in range(N):
        for j in range(n):
            C_t_x += indicator_function(x, X[i,j])
            print(x, X[i,j])
    C_t_x /= N * 0.1 ###0.1 end time (n * T_s)
    return C_t_x
n = 10
N = 5
X = torch.randn([N,n,2])
compute_Ct(torch.tensor([0.1,0.5]), X)

tensor([0.1000, 0.5000]) tensor([ 0.9456, -0.6218])
tensor([0.1000, 0.5000]) tensor([0.1495, 1.4365])
tensor([0.1000, 0.5000]) tensor([-0.1806,  0.0332])
tensor([0.1000, 0.5000]) tensor([ 0.3768, -1.9320])
tensor([0.1000, 0.5000]) tensor([-1.1082,  0.0139])
tensor([0.1000, 0.5000]) tensor([0.3874, 0.9722])
tensor([0.1000, 0.5000]) tensor([2.1498, 0.6019])
tensor([0.1000, 0.5000]) tensor([-0.5226,  1.1829])
tensor([0.1000, 0.5000]) tensor([-0.6185, -0.0812])
tensor([0.1000, 0.5000]) tensor([-0.0319,  1.3964])
tensor([0.1000, 0.5000]) tensor([0.5189, 0.2758])
tensor([0.1000, 0.5000]) tensor([-0.3546,  1.2036])
tensor([0.1000, 0.5000]) tensor([0.5321, 1.4455])
tensor([0.1000, 0.5000]) tensor([ 0.9133, -1.1659])
tensor([0.1000, 0.5000]) tensor([-1.8276, -0.5447])
tensor([0.1000, 0.5000]) tensor([-0.8559,  0.8936])
tensor([0.1000, 0.5000]) tensor([ 0.5434, -0.1795])
tensor([0.1000, 0.5000]) tensor([-1.8175,  1.7174])
tensor([0.1000, 0.5000]) tensor([ 0.2254, -1.1306])
tensor([0.1000, 0.5000

tensor(0.)